In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Imports

In [0]:
import pandas as pd
import argparse
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import requests
from IPython.core.debugger import set_trace
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
KEY_FILE_LOCATION = '/content/drive/My Drive/Google Analytics API/aq-data-pull-7d38c6520d33.json'
SERVICE_ACCOUNT_EMAIL = 'jack-aq-data@aq-data-pull.iam.gserviceaccount.com'
import warnings
warnings.simplefilter('ignore')

# Functions used to call Google API and Google Analytics variables

In [0]:
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION, scopes=SCOPES)
    http = credentials.authorize(httplib2.Http())
    # Build the service object.
    analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI,cache_discovery=False)
    return analytics

def print_response(response,date):
    data = response.get('reports', [])[0].get('data', {}).get('rows', [])
    df = pd.DataFrame(data)
    df1 = df.copy()
    try:
        l1 = df1.iloc[0][1]
    except:
        return pd.DataFrame(columns=[date,'dimensions'])
    d1 = l1[0]
    l2 = list(d1.values())
    l3 = l2[0]
    l4 = l3[0]
    df1['metrics'] = df1.iloc[:,1].apply(lambda x: x[0]).apply(lambda x: list(x.values())[0]).apply(lambda x: x[0])
    df1['dimensions'] = df1.iloc[:,0].apply(lambda x: x[0])
    df1 = df1.rename(columns={"metrics": date})
    return df1
def get_report(startDate,endDate,segment,view,pages,analytics=initialize_analyticsreporting(),page_limit=0):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
    if pages==1:
        if page_limit==0:
            print('Please enter pagelimit>0 to use with pages=1')
            return
        else:
            body={
              'reportRequests': [
                  {
                      'viewId': view,
                      'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
                      'metrics': [
                          {'expression': 'ga:pageviews'},
                      ],
                      "dimensions": [
                          {"name": "ga:landingPagePath"},
                          { "name": "ga:segment" }
                      ],
                      "segments": [
                          {'segmentId': segment},
                      ],
                      "pageSize": page_limit,
                      "orderBys": [
                        {
                          "fieldName": "ga:pageviews",
                          "sortOrder": "DESCENDING"
                              }
                      ]
            }]}
    else:
        body={
              'reportRequests': [
                  {
                      'viewId': view,
                      'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
                      'metrics': [
                          {'expression': 'ga:users'},
                      ],
                      "dimensions": [
                          { "name": "ga:segment" }
                      ],
                      "segments": [
                          {'segmentId': segment},
                      ]                              
              
          }]}
    return analytics.reports().batchGet(body=body).execute()

def date_range(start_month,start_year,end_month,end_year):
    
    def month_range(month,year):
        y = str(year)
        if month<10:
            m = '0' + str(month)
        else:
            m = str(month)
        d1 = '01'
        thirty = [9,4,6,11,2]
        if month in thirty:
            if month == 2:
                d_end = '28'
            else:
                d_end = '30'
        else:
            d_end = '31'
        start_string = y+'-'+m+'-'+d1
        end_string = y+'-'+m+'-'+d_end
        return [start_string,end_string]

    m = start_month
    l1 = []
    if start_year == end_year:
        for i in list(range(start_month,end_month+1,1)):
            l1.append(month_range(i,start_year))
        return l1
    for i in list(range(m,13,1)):
        l1.append(month_range(i,start_year))
    
    new_year = start_year+1
    while new_year != end_year:
        for i in list(range(1,13,1)):
            l1.append(month_range(i,new_year))
        new_year+=1
    for i in list(range(1,end_month+1,1)):
        l1.append(month_range(i,new_year))
    return l1                    


def monthly_site_data(dict_of_segments,dates,view): #dictionary with keys=segment name and values=segment definition
    
    def one_dataframe(segment,segment_name,dates,view):   
        master_df = pd.DataFrame()
        for i in dates:
            start = i[0]
            end = i[1]
            df = print_response(get_report(start,end,segment,view,pages=0),start)
            master_df[start] = df[start]
        df1 = master_df.transpose().rename(columns={0:segment_name})
        return df1
    
    if len(dict_of_segments) == 0:
        print('Need at least one arg')
        return
    df1 = pd.DataFrame()
    df_list = []
    if len(dict_of_segments) != 0:
        for ar in dict_of_segments:
            s1 = dict_of_segments[ar]
            s2 = ar
            df_this = one_dataframe(s1,s2,dates,view)
            df1[s2] = df_this[s2]
    return df1

def monthly_page_data(segment,view,page_limit,dates):
    counter = 0
    for months in dates:
        start = months[0]
        end = months[1]        
        df = print_response(get_report(start,end,segment,view,pages=1,page_limit=page_limit),start)
        df.set_index('dimensions',inplace=True)
        if counter == 0:
            master_df = df.copy()
            counter = 1
        else:
            master_df[start] = df[start]
            not_in = []
            for i in list(df.index):
                if i not in list(master_df.index):
                    not_in.append(i)
            master_df = master_df.append(df.loc[not_in],sort=False)
    master_df = master_df.fillna(0).astype(int)
    list_of_index = list(master_df.index)
    no_question_marks = []
    for i in list_of_index:
        if '?' in i:
            no_question_marks.append(i.split('?')[0])
        else:
            no_question_marks.append(i)
    master_df = master_df.rename(index=dict(zip(list_of_index,no_question_marks)))
    master_df.reset_index(inplace=True)
    master_df = master_df.groupby('dimensions',axis='index').sum()
    return master_df

def get_link(body):
    split1 = str(body).split('/')
    if len(split1) == 4:
        domain = str(body).split('/')[-2]
    else:
        domain = 'ibd.netitdontmatter'
    split = str(body).split(domain)
    if len(split)<2:
        return float('NaN')
    else:
        if len(split) == 2:
            return str(domain)+split[1]
        else:
            return 'Data Error: Too Many Splits'
def ga_link(link):
    response = requests.get(link, verify=False)
    url = response.url
    part = '/'.join(url.split('/')[3:])
    full = '/'+part
    return full

In [0]:
#View IDs
VIEW_ID_PSO = 'ga:122434023'
VIEW_ID_PSA = 'ga:129870027'
VIEW_ID_RA = 'ga:63773775'
VIEW_ID_AS = 'ga:188900143'
VIEW_ID_IBD = 'ga:115327156'
VIEW_ID_MS = 'ga:62646096'
VIEW_ID_T2D = 'ga:71807823'
dict_of_view_ids = {}
dict_of_view_ids['PsA'] = VIEW_ID_PSA
dict_of_view_ids['RA'] = VIEW_ID_RA
dict_of_view_ids['PsO'] = VIEW_ID_PSO
dict_of_view_ids['IBD'] = VIEW_ID_IBD
dict_of_view_ids['AS'] = VIEW_ID_AS
dict_of_view_ids['MS'] = VIEW_ID_MS
dict_of_view_ids['T2D'] = VIEW_ID_T2D

#Used for crohns only. Page restrictions and no in app traffic
crohns_cap = 'sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);condition::ga:pagePath=@/caregiver/a-husbands-perspective/,ga:pagePath=@/living/lifestyle-change-saved-life/,ga:pagePath=@/living/intuition/,ga:pagePath=@/living/5-tips-coping-high-school/,ga:pagePath=@/living/flare-ice-hack/,ga:pagePath=@/nutrition/vegan-crohns/,ga:pagePath=@/living/why-not-participate-study/,ga:pagePath=@/living/plan-life-routine/,ga:pagePath=@/nutrition/crohns-and-the-fodmap-elimination-diet/,ga:pagePath=@/living/college-crohns/,ga:pagePath=@/living/elizabeth-letter-to-self/,ga:pagePath=@/living/crohns-disease-vulva/,ga:pagePath=@/living/julie-letter-to-self/,ga:pagePath=@/living/life-hacks-living-with-fistula/,ga:pagePath=@/living/teeth-decay/,ga:pagePath=@/living/jenna-letter-to-self/,ga:pagePath=@/living/paul-letter-to-self/,ga:pagePath=@/living/colonoscopy-much-more-with-ibd/,ga:pagePath=@/living/crohns-disease-women-genitalia/,ga:pagePath=@/living/fear-passing-cd-on/' #Crohns cap performs signifcantly better
crohns_cap_proxy = 'users::sequence::ga:sessions>1;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:country=@United States;condition::ga:pagePath[]/caregiver/a-husbands-perspective/|/living/lifestyle-change-saved-life/|/living/intuition/|/living/5-tips-coping-high-school/|/living/flare-ice-hack/|/nutrition/vegan-crohns/|/living/why-not-participate-study/|/living/plan-life-routine/|/nutrition/crohns-and-the-fodmap-elimination-diet/|/living/college-crohns/;condition::ga:pagePath[]/living/elizabeth-letter-to-self/|/living/crohns-disease-vulva/|/living/julie-letter-to-self/|/living/life-hacks-living-with-fistula/|/living/teeth-decay/|/living/jenna-letter-to-self/|/living/paul-letter-to-self/|/living/colonoscopy-much-more-with-ibd/|/living/crohns-disease-women-genitalia/|/living/fear-passing-cd-on/'
#crohns_cap_proxy = 'users::sequence::ga:sessions>1;ga:sessionDuration>30,ga:pageviews>3;->>ga:pagePath=@/caregiver/a-husbands-perspective/,ga:pagePath=@/living/lifestyle-change-saved-life/,ga:pagePath=@/living/intuition/,ga:pagePath=@/living/5-tips-coping-high-school/,ga:pagePath=@/living/flare-ice-hack/,ga:pagePath=@/nutrition/vegan-crohns/,ga:pagePath=@/living/why-not-participate-study/,ga:pagePath=@/living/plan-life-routine/,ga:pagePath=@/nutrition/crohns-and-the-fodmap-elimination-diet/,ga:pagePath=@/living/college-crohns/,ga:pagePath=@/living/elizabeth-letter-to-self/,ga:pagePath=@/living/crohns-disease-vulva/,ga:pagePath=@/living/julie-letter-to-self/,ga:pagePath=@/living/life-hacks-living-with-fistula/,ga:pagePath=@/living/teeth-decay/,ga:pagePath=@/living/jenna-letter-to-self/,ga:pagePath=@/living/paul-letter-to-self/,ga:pagePath=@/living/colonoscopy-much-more-with-ibd/,ga:pagePath=@/living/crohns-disease-women-genitalia/,ga:pagePath=@/living/fear-passing-cd-on/'
#crohns_cap_proxy = 'users::sequence::ga:sessions>1;ga:pageviews>3,ga:sessionDurationBucket>30;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);condition::ga:pagePath=@/caregiver/a-husbands-perspective/,ga:pagePath=@/living/lifestyle-change-saved-life/,ga:pagePath=@/living/intuition/,ga:pagePath=@/living/5-tips-coping-high-school/,ga:pagePath=@/living/flare-ice-hack/,ga:pagePath=@/nutrition/vegan-crohns/,ga:pagePath=@/living/why-not-participate-study/,ga:pagePath=@/living/plan-life-routine/,ga:pagePath=@/nutrition/crohns-and-the-fodmap-elimination-diet/,ga:pagePath=@/living/college-crohns/,ga:pagePath=@/living/elizabeth-letter-to-self/,ga:pagePath=@/living/crohns-disease-vulva/,ga:pagePath=@/living/julie-letter-to-self/,ga:pagePath=@/living/life-hacks-living-with-fistula/,ga:pagePath=@/living/teeth-decay/,ga:pagePath=@/living/jenna-letter-to-self/,ga:pagePath=@/living/paul-letter-to-self/,ga:pagePath=@/living/colonoscopy-much-more-with-ibd/,ga:pagePath=@/living/crohns-disease-women-genitalia/,ga:pagePath=@/living/fear-passing-cd-on/'
#Restrictions: Everything EXCEPT FB in app traffic
cs_total = 'sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
all_total = 'sessions::condition::ga:country=@United States'

#Restrictions: Only INCLUDE facebook in app traffic. (Used to compare the in app traffic to non-in app traffic)
total_just_fb = 'sessions::condition::ga:country=@United States;condition::ga:browser=@Safari (in-app)'
proxy_just_fb = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:country=@United States;condition::ga:browser=@Safari (in-app)'

#Restrictions: Facebook only. No FB in app traffic. 
total_cs_fb = 'users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy_cs_fb = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy_cs_fb_pvs = 'sessions::sequence::ga:pageviews>1;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy_cs_fb_ssns = 'users::sequence::ga:sessions>=2;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'

#Restrictions: Promoted FB only. No in app traffic.
promoted_fb = 'users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);sessions::condition::ga:sourceMedium=@facebook.com / promoted'
proxy_promoted = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);sessions::condition::ga:sourceMedium=@facebook.com / promoted'

total_fb = 'sessions::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States'
proxy_fb = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States'

bd_facebook_proxy = 'users::sequence::ga:sessions>=2,ga:sessionDuration>120,ga:pageviews>2;sessions::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States'
bd_site_proxy = 'users::sequence::ga:sessions>=2,ga:sessionDuration>120,ga:pageviews>2;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'


total_organic = 'sessions::condition::ga:channelGrouping=@Organic Search'
proxy_organic = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:channelGrouping=@Organic Search'

In [0]:
dol_crohns = {}
dol_crohns['Proxy FB'] = proxy_cs_fb
dol_crohns['Total FB'] = total_cs_fb
dol_crohns['All FB Users'] = total_fb
dol_crohns['Site Proxy'] = proxy
dol_crohns['Site Total'] = cs_total
dol_crohns['PVS FB'] = proxy_cs_fb_pvs
dol_crohns['SSN FB'] = proxy_cs_fb_ssns

# Helper functions and predictions class

In [0]:
def prep_old_fb(ofacebook_data):
    ofacebook_data_all = ofacebook_data.groupby('Month').sum()
    ofacebook_data_all = ofacebook_data_all.rename(columns={'Cost per Result': 'CPC (All)'})
    return ofacebook_data_all
def prep_new_fb(nfacebook_data,drop='y'):
    if drop=='y':
        nfacebook_data = nfacebook_data.drop(0,axis=0)
    nfacebook_data_all = nfacebook_data.groupby('Month').sum()
    return nfacebook_data_all
def format_regression_data(facebook_data_all,ga_data,aq_df=None,want_aq='yes'):
    facebook_data_all = facebook_data_all.dropna(axis='columns',how='any')
    if want_aq=='yes':
        if aq_df.empty:
            print('Enter AQ Data')
        else:
            facebook_data_all['AQ'] = list(aq_df['AQ'])
            facebook_data_all['Date'] = list(aq_df['Date'])
            facebook_data_all = facebook_data_all.set_index('Date')
    facebook_data_all['PP'] = list(ga_data['Percent'])
    facebook_data_all['CpS'] = facebook_data_all['Post Shares']/facebook_data_all['Amount Spent (USD)']
    facebook_data_all['Total Users'] = list(ga_data['Total FB'])
    facebook_data_all['Cap %'] = list(ga_data['Cap %'])
    facebook_data_all['Frequency'] = facebook_data_all['Impressions']/facebook_data_all['Reach']
    facebook_data_all['Imp/Cost'] = facebook_data_all['Impressions']/facebook_data_all['Amount Spent (USD)']
    facebook_data_all['3 Mo Imp'] = facebook_data_all['Impressions'].rolling(3,min_periods=1).sum()
    return facebook_data_all
def format_ga_data(ga_data):
    ga_data = ga_data.astype(int)
    ga_data['Percent'] = ga_data['Proxy FB']/ga_data['Total FB']
    ga_data['Cap %'] = ga_data['Total FB']/ga_data['All FB Users']
    return ga_data
def one(df,params):
    pop = list(np.random.choice(len(df),2,replace=False))
    currentdf = df.drop(pop,axis='index')
    model1 = ols.fit(np.asarray(currentdf[params]), currentdf['AQ'])
    pred1 = model1.predict(np.asarray(df.iloc[pop][params]))
    coefs = model1.coef_
    inter = model1.intercept_
    se = mean_absolute_error(df.iloc[pop]['AQ'],pred1)
    r2 = r2_score(df.iloc[pop]['AQ'],pred1)
    df['Pred'] = [float('NaN') for x in range(len(df))]
    for i in range(len(pop)):
        df['Pred'].iat[pop[i]] = pred1[i] 
    return(coefs,inter,se,r2,df.loc[~df['Pred'].isnull()]['Pred'])
def bootstrap(df,params,dates):
    result = pd.DataFrame(index=df.index)
    c1s = []
    c2s = []
    c3s = []
    linter = []
    ses = []
    r2s = []
    for i in range(1000):        
        one1 = one(df,params)
        result[str(i)] = one1[-1]
        c1s.append(one1[0][0])
        c2s.append(one1[0][1])
        c3s.append(one1[0][2])
        linter.append(one1[1])
        ses.append(one1[2])
        r2s.append(one1[3])
    c1s.sort()
    lowc1 = c1s[250]
    highc1 = c1s[750]
    
    c2s.sort()
    lowc2 = c2s[250]
    highc2 = c2s[750]
    
    c3s.sort()
    lowc3 = c3s[250]
    highc3 = c3s[750]
    
    linter.sort()
    lint = linter[250]
    uint = linter[750]
    
    me = np.asarray(ses).mean()
    r2 = np.asarray(r2s).mean()
    
    result = result.transpose()
    highs = []
    lows = []
    meds = []
    for i in result.columns:
        result1 = result.sort_values(i,ascending=False)[i] #Highest at top
        result1 = result1.dropna(axis=0)
        q = len(result1)//4
        high = result1.iat[q]
        low = result1.iat[q*3]
        med = result1.iat[q*2]
        highs.append(high)
        lows.append(low)
        meds.append(med)
    result = result.transpose()
    result['Low'] = lows
    result['High'] = highs
    result['Med'] = meds
    lowcoefs = [lowc1,lowc2,lowc3,lint]
    highcoefs = [highc1,highc2,highc3,uint]
    
    return(lowcoefs,highcoefs,me,r2,result[['Low','Med','High']])
def avg_model(array):
    pp = .3
    cap = .25
    imp_cost = 2.25E-4
    out = pp*array[0]+cap*array[1]+imp_cost*array[2]
    return out

def plot_dates(startm,starty,endm,endy):
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    index = startm-1
    dates = []
    start = True
    if starty == endy:
        for i in range(index,endm):
            dates.append(str(starty)+'-'+months[i])
        return dates
    for i in range(index,12):
        dates.append(str(starty)+'-'+months[i])
    starty+=1
    while starty<endy:
        for i in range(12):
            dates.append(str(starty)+'-'+months[i])
        starty+=1
    for i in range(endm):
        dates.append(str(starty)+'-'+months[i])
    return dates

import plotly.graph_objects as go
from plotly.subplots import make_subplots

class predictions:
    def __init__(self,old_data,new_data,condition,dates):
        self.fb_data = pd.concat([prep_old_fb(old_data),prep_new_fb(new_data)],axis=0)
        self.condition = condition
        if self.condition=='CD':
            self.view_id = dict_of_view_ids['IBD']
        else:
            self.view_id = dict_of_view_ids[condition]
        self.dates = dates
        self.ga_data = None
        self.reg_data = None
        self.features = None
        self.poly_reg_data = None
        self.aq = None
        self.r2_score = None
        self.mean_error = None
        self.model = None
        self.preds = None
        self.lowcoef = None
        self.highcoef = None
        self.bs_error = None
        self.bs_r2 = None
        self.prediction_bs_ci = None
        self.plot = None
        self.last_two_r2 = None
        self.last_two_error = None
        if self.condition in ['CD','PsA','PsO','RA']:
            if self.condition=='CD':
                self.cap_rate = .38
                self.year_goal = 18800
                self.month_goal = self.year_goal/12
            elif self.condition=='PsA':
                self.cap_rate = .65
                self.year_goal = 22156
                self.month_goal = self.year_goal/12
            elif self.condition=='PsO':
                self.cap_rate= .65
                self.year_goal = 26000
                self.month_goal = self.year_goal/12
            elif self.condition=='RA':
                self.cap_rate=.48
                self.year_goal = 86843
                self.month_goal = self.year_goal/12
    def get_ga_data(self,dict_of_segments):
        self.segments = dict_of_segments
        self.ga_data = format_ga_data(monthly_site_data(dict_of_segments,self.dates,self.view_id))
    def regression_data(self,aq_data,features):
        self.reg_data = format_regression_data(self.fb_data,self.ga_data,aq_df=aq_data).dropna(axis=0)
        self.features = features
    def run_regression(self,skip_mons=0):
        regression = self.reg_data
        regression = regression.iloc[skip_mons:]
        self.plot_dates = list(regression.index)
        poly = PolynomialFeatures(5).fit(regression)
        regression_poly = pd.DataFrame(poly.transform(regression),columns=poly.get_feature_names(regression.columns))
        regression = regression_poly.copy()
        self.poly_reg_data = regression
        ols = LinearRegression()
        X = regression[self.features]
        y = regression['AQ']
        self.aq = y
        model = ols.fit(X,y)
        pred = model.predict(X)
        self.r2_score = r2_score(y,pred)
        self.mean_error = mean_absolute_error(y,pred)
        self.model = model
        self.preds = pred
        self.last_score = avg_model(np.asarray(X.iloc[-1:])[0])
    def last_2_performance(self):
        X_train = self.poly_reg_data[self.features].iloc[:-2]
        X_test = self.poly_reg_data[self.features].iloc[-2:]
        y_train = self.aq.iloc[:-2]
        y_test = self.aq.iloc[-2:]
        ols = LinearRegression()
        model = ols.fit(X_train,y_train)
        pred = model.predict(X_test)
        self.last_two_r2 = r2_score(y_test,pred)
        self.last_two_error = mean_absolute_error(y_test,pred)
        self.last_two_pred = pred
        self.last_two_actual = y_test
    def bootstrap_stats(self):
        self.lowcoef,self.highcoef,self.bs_error,self.bs_r2,self.prediction_bs_ci = bootstrap(self.poly_reg_data,self.features,self.dates)
    def plot_results(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.plot_dates,y=self.aq,mode='markers',))
        fig.add_trace(go.Scatter(x=self.plot_dates,y=self.preds,mode='lines',line_color='rgba(13,170,201,1)'))
        high = list(self.preds+self.mean_error*1.5)
        low = list(self.preds-self.mean_error*1.5)
        fig.add_trace(go.Scatter(x=self.plot_dates+self.plot_dates[::-1],y=high+low[::-1],fill='toself',fillcolor='rgba(13,170,201,.15)',line_color='rgba(255,255,255,0)',showlegend=False))
        fig.update_layout(title=self.condition+' AQ',xaxis_title='Month',yaxis_title='cS Audience Quality',showlegend=False)
        self.plot = fig
    def make_predictions(self,new_df,dates):
        new_ga_data = format_ga_data(monthly_site_data(self.segments,dates,self.view_id))
        new_fb_data = prep_new_fb(new_df,drop='n')
        reg_data = format_regression_data(new_fb_data,new_ga_data,want_aq='no')
        self.new_pred_reg_data = reg_data
        pred = self.model.predict(reg_data[self.features])
        self.new_preds = pred
        self.new_preds_upper = pred+self.bs_error*1.5
        self.new_preds_lower = pred-self.bs_error*1.5
        scores = []
        for i in range(len(reg_data)):
            scores.append(avg_model(np.asarray(reg_data.iloc[i][self.features])))
        self.new_scores = scores
        for i in self.new_scores:
            if (i-self.last_score)<-.02:
                print('Warning: Model average shows potentially steep decline')
    def update_plot(self,plot_dates):
        self.pred_dates = plot_dates
        fig = self.plot
        plotd = [self.preds[-1]]
        for i in self.new_preds:
            plotd.append(i)
        upper = []
        for i in range(len(plotd)):
            if i==0:
                upper.append(plotd[i]+self.mean_error*1.5)
            else:
                upper.append(plotd[i]+self.bs_error*1.5)
        lower = []
        for i in range(len(plotd)):
            if i==0:
                lower.append(plotd[i]-self.mean_error*1.5)
            else:
                lower.append(plotd[i]-self.bs_error*1.5)
        lower = lower[::-1]
        fig.add_trace(go.Scatter(x=plot_dates,y=plotd,line=dict(color='rgba(205,10,29,1)',dash='dash'),mode='lines'))
        fig.add_trace(go.Scatter(x=plot_dates+plot_dates[::-1],y=upper+lower,fill='toself',line_color='rgba(255,255,255,0)',fillcolor='rgba(193,66,66,.3)'))
        self.plot = fig
    def create_humira_plot(self,uvs,duvs):
        self.duvs = duvs
        x = self.pred_dates[1:]
        prev_dates = self.plot_dates[-6:]
        prev_duvs = self.duvs[-6:]
        prev_aq = self.aq[-6:]
        prev_aq_q1 = round(self.aq[-12:-9].mean(),3)
        prev_aq_q2 = round(self.aq[-9:-6].mean(),3)
        prev_duvs_q1 = round(self.duvs[-12:-9].mean(),0)
        prev_duvs_q2 = round(self.duvs[-9:-6].mean(),0)
        
        previous_duv_graph = [prev_duvs_q1,prev_duvs_q2]+list(prev_duvs)
        previous_aq_graph = [prev_aq_q1,prev_aq_q2]+list(prev_aq)
        index = ['Q1 Mean','Q2 Mean']+self.plot_dates[-6:]

        bar = self.new_preds*uvs*self.cap_rate
        self.predicted_duvs = bar
        bar_errors = (self.bs_error*1.65)*uvs*self.cap_rate
        self.predicted_duvs_upper = bar+bar_errors
        self.predicted_duvs_lower = bar-bar_errors
        bar_errors = [round(x,0) for x in bar_errors]
        bar = [round(x,0) for x in bar]
        line = self.new_preds
        line = [round(x,3) for x in line]
        line.insert(0,self.aq.iloc[-1])
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])        
        pred_scatter_index =[index[-1]]+x 
        fig.add_trace(go.Bar(x=index,y=previous_duv_graph,name='Previous DUVs',marker_color='rgba(140,140,140,1)'),secondary_y=False)
        fig.add_trace(go.Bar(x=x,y=bar,name='Predicted DUVs',error_y=dict(type='data',array=bar_errors),marker_color='rgba(77,162,94,1)'),secondary_y=False)
        fig.add_trace(go.Scatter(x=pred_scatter_index,y=line,name='Predicted AQ',error_y=dict(type='data',array=[0]+[round(self.bs_error*1.65,3) for x in range(len(line)-1)]),line=dict(width=4,dash='dash',color='rgba(255,166,0,1)')),secondary_y=True)
        fig.add_trace(go.Scatter(x=index,y=previous_aq_graph,line=dict(width=4,color='rgba(2,80,92,1)'),name='Previous AQ'),secondary_y=True)
        fig.update_yaxes(title_text='AQ',range=[0,self.aq.max()*2],secondary_y=True)
        fig.update_yaxes(title_text='DUVs',range=[0,self.month_goal*2],secondary_y=False)
        fig.add_shape(type="line",xref="paper",yref="y",x0=0,y0=self.month_goal,x1=.94,
                    y1=self.month_goal,line=dict(color='rgba(120,120,120,1)',width=1,dash='dash'))
        fig.update_layout(title_text=self.condition+' Projections')
        fig.update_layout(annotations=[dict(xref='paper',yref='y',x=0,y=self.month_goal,text='Monthly DUV Goal',showarrow=False,xanchor='left',yanchor='bottom')])
        self.humira_plot = fig

In [0]:
root_dir = '/content/drive/My Drive/Google Analytics API/AQ/ML/'
df=pd.read_excel(root_dir+'/Jan-Mar New Data.xlsx').drop(0,axis='index')
def get_new_dfs(df):
    new_cd_df = df.loc[df['Account Name'].str.contains('Inf')]
    new_ra_df = df.loc[df['Account Name'].str.contains('Rhe')]
    new_psa_df = df.loc[df['Account Name'].str.contains('Psoriat')]
    new_pso_df = df.loc[df['Account Name'].str.contains('Psoriasis')]
    return new_cd_df,new_ra_df,new_psa_df,new_pso_df
new_cd_df = get_new_dfs(df)[0]
new_ra_df = get_new_dfs(df)[1]
new_psa_df = get_new_dfs(df)[2]
new_pso_df = get_new_dfs(df)[3]

In [0]:
df_uvs = pd.read_excel(root_dir+'Jan-Mar DFA UV Data.xlsx')
df_uvs = df_uvs.rename(columns={'Unnamed: 0':'Condition'})
df_uvs = df_uvs.set_index('Condition')

In [0]:
duvs = pd.read_excel(root_dir+'2019 DUV Data.xlsx')
duvs.set_index('Condition',inplace=True)

# Predictions

In [14]:
ofacebook_data_cd = pd.read_excel(root_dir+'IBD Facebook Data Old.xlsx')
nfacebook_data_cd = pd.read_excel(root_dir+'IBD Facebook Data.xlsx')
aq_df_cd = pd.read_csv(root_dir+'comScore Data/Humira AQ CD.csv')

cd_predictions = predictions(ofacebook_data_cd,nfacebook_data_cd,'CD',date_range(1,2018,12,2019))
cd_predictions.get_ga_data(dol_crohns)

ols = LinearRegression()
features = ['PP','Cap %','Imp/Cost']
cd_predictions.regression_data(aq_df_cd.iloc[22:],features)
cd_predictions.run_regression(skip_mons=8)
cd_predictions.r2_score

cd_predictions.bootstrap_stats()
cd_predictions.plot_results()
cd_predictions.make_predictions(new_cd_df,date_range(1,2020,3,2020))

uvs = np.asarray(list(df_uvs.loc['IBD']))
cd_duvs = duvs.loc['CD']
cd_predictions.update_plot(plot_dates(12,19,3,20))
cd_predictions.create_humira_plot(uvs,cd_duvs)
cd_predictions.humira_plot

In [0]:
plotly.offline.plot(cd_predictions.humira_plot,filename='Humira CD.html',auto_open=False)

'Humira CD.html'

In [15]:
ofacebook_data_psa = pd.read_excel(root_dir+'PsA Facebook Data Old.xlsx')
nfacebook_data_psa = pd.read_excel(root_dir+'PsA Facebook Data New.xlsx')
aq_df_psa = pd.read_csv(root_dir+'comScore Data/PsA Humira AQ.csv')

psa_predictions = predictions(ofacebook_data_psa,nfacebook_data_psa,'PsA',date_range(6,2018,12,2019))
psa_predictions.get_ga_data(dol_crohns)

features = ['PP','Cap %','Imp/Cost']
psa_predictions.regression_data(aq_df_psa,features)
psa_predictions.run_regression()

psa_predictions.bootstrap_stats()
psa_predictions.plot_results()

uvs = np.asarray(list(df_uvs.loc['PsA']))
psa_duvs = duvs.loc['PsA']
psa_predictions.make_predictions(new_psa_df,date_range(1,2020,3,2020))
psa_predictions.update_plot(plot_dates(12,19,3,20))
psa_predictions.create_humira_plot(uvs,psa_duvs)
psa_predictions.humira_plot


In [0]:
plotly.offline.plot(psa_predictions.humira_plot,filename='Humira PsA.html',auto_open=False)

'Humira PsA.html'

In [16]:
ofacebook_data_pso = pd.read_excel(root_dir+'PsO Facebook Data Old.xlsx')
nfacebook_data_pso = pd.read_excel(root_dir+'PsO Facebook Data New.xlsx')
aq_df_pso = pd.read_csv(root_dir+'comScore Data/PsO Humira AQ.csv')

pso_predictions = predictions(ofacebook_data_pso,nfacebook_data_pso,'PsO',date_range(4,2018,12,2019))
pso_predictions.get_ga_data(dol_crohns)

pso_predictions.regression_data(aq_df_pso.iloc[19:],features=['PP','Cap %','Imp/Cost'])
pso_predictions.run_regression(skip_mons=0)

pso_predictions.bootstrap_stats()

pso_predictions.plot_results()
pso_predictions.plot

pso_predictions.make_predictions(new_pso_df,date_range(1,2020,3,2020))
pso_predictions.update_plot(plot_dates(12,19,3,20))

uvs = np.asarray(list(df_uvs.loc['PsO']))
pso_duvs = duvs.loc['PsO']
pso_predictions.create_humira_plot(uvs,pso_duvs)
pso_predictions.humira_plot

In [0]:
plotly.offline.plot(pso_predictions.humira_plot,filename='Humira PsO.html',auto_open=False)

'Humira PsO.html'

In [17]:
ofacebook_data_ra = pd.read_excel(root_dir+'RA Facebook Data Old.xlsx')
nfacebook_data_ra = pd.read_excel(root_dir+'RA Facebook Data New.xlsx')
aq_df_ra = pd.read_csv(root_dir+'comScore Data/Humira AQ RA.csv')

ra_predictions = predictions(ofacebook_data_ra,nfacebook_data_ra,'RA',date_range(1,2018,12,2019))
ra_predictions.get_ga_data(dol_crohns)
aq_df_ra = pd.read_csv(root_dir+'comScore Data/Humira AQ RA.csv')

ra_predictions.regression_data(aq_df_ra.iloc[25:],features=['PP','Cap %','Imp/Cost'])
ra_predictions.run_regression(skip_mons=4)
ols = LinearRegression()
ra_predictions.bootstrap_stats()
ra_predictions.r2_score

ra_predictions.plot_results()

ra_predictions.make_predictions(new_ra_df,date_range(1,2020,3,2020))
ra_predictions.update_plot(plot_dates(12,19,3,20))

uvs = np.asarray(list(df_uvs.loc['RA']))
ra_duvs = duvs.loc['RA']
ra_predictions.create_humira_plot(uvs,ra_duvs)
ra_predictions.humira_plot

In [0]:
import plotly
plotly.offline.plot(ra_predictions.humira_plot,filename='Humira RA.html',auto_open=False)

'Humira RA.html'

# All-Humira Historic Plot
- Generate a plot with 4 subplots for each of the measurable humiras showing the trend over time of AQ

In [0]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("CD", "RA", "PsO", "PsA"))
fig.add_trace(go.Scatter(x=cd_predictions_long.plot_dates,y=cd_predictions_long.preds), row=1, col=1)
fig.add_trace(go.Scatter(x = ra_predictions.plot_dates,y=ra_predictions.preds), row=1, col=2)
fig.add_trace(go.Scatter(x = pso_predictions.plot_dates, y = pso_predictions.preds), row=2,col=1)
fig.add_trace(go.Scatter(x = psa_predictions.plot_dates,y=psa_predictions.preds,mode='lines'),row=2,col=2)

fig.add_trace(go.Scatter(x=cd_predictions_long.plot_dates,y=cd_predictions_long.aq,mode='markers'), row=1, col=1)
fig.add_trace(go.Scatter(x = ra_predictions.plot_dates,y=ra_predictions.aq,mode='markers'), row=1, col=2)
fig.add_trace(go.Scatter(x = pso_predictions.plot_dates, y = pso_predictions.aq,mode='markers'), row=2,col=1)
fig.add_trace(go.Scatter(x = psa_predictions.plot_dates,y=psa_predictions.aq,mode='markers'),row=2,col=2)

fig.update_layout(showlegend=False)
plotly.offline.plot(fig,filename='AQ.html',auto_open=False)

'AQ.html'

# Create DUV pacing excel sheet

In [0]:
def create_duv_pacing_excel(month):
  index = ['Yearly DUV Goal','Monthly DUV Goal','Estimated Pacing Through March',
          'Low Pacing Through March','High Pacing Through March','Percent of Year Completed',
          'Estimated Percent Pacing','Low Percent Pacing','DUV Pacing Gap','Percent Pacing Gap']
  lop = [cd_predictions,psa_predictions,pso_predictions,ra_predictions]
  rows = []
  for i in lop:
      rows.append([i.year_goal,round(i.month_goal,0),round(i.predicted_duvs.sum(),0),
                  round(i.predicted_duvs_lower.sum(),0),
                round(i.predicted_duvs_upper.sum(),0),round(month/12,3),
                  round(i.predicted_duvs.sum()/i.year_goal,3),
                  round(i.predicted_duvs_lower.sum()/i.year_goal,3),
                  round(i.predicted_duvs.sum()-(i.month_goal*month),0),
                  round((i.predicted_duvs.sum()/i.year_goal-(month/12)),3)])
  df = pd.DataFrame(rows,columns=index,index=['CD','PsA','PsO','RA'])
  df.to_excel('DUV Pacing.xlsx')
  return df

# Process for Deep Learning

In [0]:
df_ra = (ra_predictions.reg_data-ra_predictions.reg_data.mean())/ra_predictions.reg_data.std()
#df_ra = ra_predictions.reg_data
df_ra = (df_ra-df_ra.shift(1)).dropna()
df_ra = df_ra.iloc[6:]
df_ra['RA'] = [1 for x in range(len(df_ra))]

df_pso = (pso_predictions.reg_data-pso_predictions.reg_data.mean())/pso_predictions.reg_data.std()
#df_pso = pso_predictions.reg_data
df_pso = (df_pso-df_pso.shift(1)).dropna()
df_pso['PsO'] = [1 for x in range(len(df_pso))]

df_psa = (psa_predictions.reg_data-psa_predictions.reg_data.mean())/psa_predictions.reg_data.std()
#df_psa = psa_predictions.reg_data
df_psa = (df_psa-df_psa.shift(1)).dropna()
df_psa['PsA'] = [1 for x in range(len(df_psa))]

df_cd = (cd_predictions.reg_data-cd_predictions.reg_data.mean())/cd_predictions.reg_data.std()
#df_cd = cd_predictions.reg_data
df_cd = (df_cd-df_cd.shift(1)).dropna()
df_cd = df_cd.iloc[6:]
df_cd['CD'] = [1 for x in range(len(df_cd))]

In [0]:
df_all = pd.concat([df_cd,df_psa,df_pso,df_ra])
def binary(x):
    if x>0:
        return 1
    else:
        return 0
df_all['Binary'] = df_all['AQ'].apply(lambda x: binary(x))

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

features = ['PP','Cap %','Imp/Cost']
ols = LogisticRegression()
X = df_all[features]
y = df_all['Binary']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.1,random_state=1)
model = ols.fit(X_train,y_train)
pred = model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.875

In [0]:
df_all = df_all.fillna(0)
df_all[['PP','Cap %','Imp/Cost','Impressions','Total Users','CD','PsA','PsO','RA','AQ']].to_csv('/Users/jackarmand/Documents/AQ/Data.csv',index=False)

In [0]:
X = cd_predictions.reg_data[features]
y = cd_predictions.reg_data['AQ']
ols = LinearRegression()
model = ols.fit(X,y)
pred = model.predict(X)
from sklearn.metrics import mean_squared_error
mean_squared_error(y,pred)

0.0006978190599934586

In [0]:
features = ['PP','Cap %','Imp/Cost']
new_df = cd_predictions.new_pred_reg_data[features]
t = cd_predictions.reg_data[features].iloc[8:]
new_df = (new_df-t.mean())/t.std()
new_df.to_csv('/Users/jackarmand/Documents/AQ/CD Pred Data.csv',index=False)

In [0]:
aq = cd_predictions.reg_data[['AQ']].iloc[8:]
aq.std()

AQ    0.011867
dtype: float64

In [0]:
pso = process_for_dl(pso_predictions,4,0)

In [0]:
print(pso[2].mean()['AQ'])
print(pso[2].std()['AQ'])

0.10858571428571431
0.01908832852070306


In [0]:
pso[0].iloc[2:].to_csv('/Users/jackarmand/Documents/AQ/PsO Data.csv',index=False)
pso[1].to_csv('/Users/jackarmand/Documents/AQ/PsO Pred Data.csv',index=False)

In [0]:
psa = process_for_dl(psa_predictions,6,0)

In [0]:
print(psa[2].mean()['AQ'])
print(psa[2].std()['AQ'])

0.08705555555555554
0.022346133294210303


In [0]:
psa[0].iloc[2:].to_csv('/Users/jackarmand/Documents/AQ/PsA Data.csv',index=False)
psa[1].to_csv('/Users/jackarmand/Documents/AQ/PsA Pred Data.csv',index=False)

In [0]:
psa[2].std()['AQ']

0.022346133294210303

In [0]:
ra = process_for_dl(ra_predictions,1,4)

In [0]:
ra[0].iloc[2:].to_csv('/Users/jackarmand/Documents/AQ/RA Data.csv',index=False)
ra[1].to_csv('/Users/jackarmand/Documents/AQ/RA Pred Data.csv',index=False)
print(ra[2].mean()['AQ'])
print(ra[2].std()['AQ'])

0.183
0.02944039473419579


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
ols = LinearRegression()
features = ['PP','Cap %','Imp/Cost']
X = psa[0].iloc[2:][features]
y = psa[0].iloc[2:]['AQ']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=50)
model = ols.fit(X_train,y_train)
pred = model.predict(X_test)
print(r2_score(y_test,pred))
print(mean_squared_error(y_test,pred))

-0.7188740337002488
0.8663650575403374


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
ols = LinearRegression()
features = ['PP','Cap %','Imp/Cost']
X = pso[0].iloc[2:][features]
y = pso[0].iloc[2:]['AQ']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=10)
model = ols.fit(X_train,y_train)
pred = model.predict(X_test)
print(r2_score(y_test,pred))
print(mean_squared_error(y_test,pred))

0.5987070940844643
0.12555399381333335


In [0]:
np.median(np.array(r2s))

0.2972970815996785

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
r2s = []
for i in range(10):
    ols = LinearRegression()
    features = ['PP','Cap %','Imp/Cost']
    X = ra[0].iloc[2:][features]
    y = ra[0].iloc[2:]['AQ']
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=i)
    model = ols.fit(X_train,y_train)
    pred = model.predict(X_test)
    r2s.append(r2_score(y_test,pred))
    print(r2_score(y_test,pred))
    print(mean_squared_error(y_test,pred))
    print('\n')

-0.22605105901567257
1.215106833507092


0.1436547467953484
0.8306082251042154


0.6497923219644992
0.3362731637634119


0.5968111748270901
0.21848932931410156


0.4056505833869365
0.9716826174079131


-0.2827894862187734
0.46213683048120174


0.3035404569503116
0.33040646762188075


-1.4523247761649607
0.6359023485194578


0.4508897052919021
0.9343094325537151


0.29105370624904536
0.9328705193366714




In [0]:
from IPython.core.debugger import set_trace

In [0]:
def process_for_dl(pred_obj,start_m,skip_mons):
    additional_df = monthly_site_data(dol_crohns,date_range(start_m,2018,12,2019),pred_obj.view_id)
    additional_df_new = monthly_site_data(dol_crohns,date_range(1,2020,2,2020),pred_obj.view_id)
    
    additional_df = additional_df.astype(int)
    additional_df['PP'] = additional_df['Proxy FB']/additional_df['Total FB']
    additional_df['SPP'] = additional_df['SSN FB']/additional_df['Total FB']
    additional_df['PVPP'] = additional_df['PVS FB']/additional_df['Total FB']
    additional_df['Site PP'] = additional_df['Site Proxy']/additional_df['Site Total']
    
    additional_df_new = additional_df_new.astype(int)
    additional_df_new['PP'] = additional_df_new['Proxy FB']/additional_df_new['Total FB']
    additional_df_new['SPP'] = additional_df_new['SSN FB']/additional_df_new['Total FB']
    additional_df_new['PVPP'] = additional_df_new['PVS FB']/additional_df_new['Total FB']
    additional_df_new['Site PP'] = additional_df_new['Site Proxy']/additional_df_new['Site Total']

    all_ga_df = pd.concat([additional_df,additional_df_new])
    all_fb_data = pd.concat([pred_obj.reg_data,pred_obj.new_pred_reg_data])
    all_fb_data['3 Mo Imp'] = all_fb_data['Impressions'].rolling(3).mean()
    all_fb_data['SPP'] = list(all_ga_df['SPP'])
    all_fb_data['PVPP'] = list(all_ga_df['PVPP'])
    all_fb_data['Site PP'] = list(all_ga_df['Site PP'])

    train = all_fb_data.iloc[:-2]
    train = train.iloc[skip_mons:]
    train_norm = (train-train.mean())/train.std()

    pred = all_fb_data.iloc[-2:]
    pred = (pred-train.mean())/train.std()

    train_norm = train_norm[['PP','Cap %','Imp/Cost','3 Mo Imp','SPP','PVPP','Site PP','AQ']]
    pred = pred[['PP','Cap %','Imp/Cost','3 Mo Imp','SPP','PVPP','Site PP']]
    
    return train_norm,pred,train
#train_norm.to_csv('/Users/jackarmand/Documents/AQ/CD Data.csv',index=False)
#pred.to_csv('/Users/jackarmand/Documents/AQ/CD Pred Data.csv',index=False)

In [0]:
additional_df = monthly_site_data(dol_crohns,date_range(1,2018,12,2019),dict_of_view_ids['IBD'])

In [0]:
additional_df_new = monthly_site_data(dol_crohns,date_range(1,2020,2,2020),dict_of_view_ids['IBD'])

In [0]:
additional_df = additional_df.astype(int)
additional_df['PP'] = additional_df['Proxy FB']/additional_df['Total FB']
additional_df['SPP'] = additional_df['SSN FB']/additional_df['Total FB']
additional_df['PVPP'] = additional_df['PVS FB']/additional_df['Total FB']
additional_df['Site PP'] = additional_df['Site Proxy']/additional_df['Site Total']

In [0]:
additional_df_new = additional_df_new.astype(int)
additional_df_new['PP'] = additional_df_new['Proxy FB']/additional_df_new['Total FB']
additional_df_new['SPP'] = additional_df_new['SSN FB']/additional_df_new['Total FB']
additional_df_new['PVPP'] = additional_df_new['PVS FB']/additional_df_new['Total FB']
additional_df_new['Site PP'] = additional_df_new['Site Proxy']/additional_df_new['Site Total']

In [0]:
all_ga_df = pd.concat([additional_df,additional_df_new])

all_fb_data = pd.concat([cd_predictions.reg_data,cd_predictions.new_pred_reg_data])
all_fb_data['3 Mo Imp'] = all_fb_data['Impressions'].rolling(3).mean()
all_fb_data['SPP'] = list(all_ga_df['SPP'])
all_fb_data['PVPP'] = list(all_ga_df['PVPP'])
all_fb_data['Site PP'] = list(all_ga_df['Site PP'])

train = all_fb_data.iloc[:-2]
train = train.iloc[4:]
train_norm = (train-train.mean())/train.std()

pred = all_fb_data.iloc[-2:]
pred = (pred-train.mean())/train.std()

train_norm = train_norm[['PP','Cap %','Imp/Cost','3 Mo Imp','SPP','PVPP','Site PP','AQ']]
pred = pred[['PP','Cap %','Imp/Cost','3 Mo Imp','SPP','PVPP','Site PP']]
train_norm.to_csv('/Users/jackarmand/Documents/AQ/CD Data.csv',index=False)
pred.to_csv('/Users/jackarmand/Documents/AQ/CD Pred Data.csv',index=False)